In [16]:
import torch
import numpy as np 
from datautil import train_set, NUM_FEATURES
from math import log2 
DATASET_SIZE = len(train_set)
bayesian_weight = [0] * NUM_FEATURES
feature_occ = [0] * NUM_FEATURES
num_p_s = [0] * NUM_FEATURES
num_n_s = [0] * NUM_FEATURES
num_p = 0
num_n = 0

In [17]:
print(DATASET_SIZE)
for i in range(DATASET_SIZE):
    fp, tox = train_set[i]
    fp = fp.detach().numpy()
    if tox == 1: num_p += 1 
    else: num_n += 1
    for j in range(NUM_FEATURES):
        if fp[j]: 
            feature_occ[j] += 1
            if tox == 1: num_p_s[j] += 1 
            else: num_n_s[j] += 1 

828


In [93]:
weight = [0] * NUM_FEATURES
THRESHOLD = 0.1
MULTIPLIER = 10
print(num_p_s)
print(num_n_s)
for i in range(NUM_FEATURES):
    weight[i] = log2((num_p_s[i] + 1) / (num_n_s[i] + 1))
    if weight[i] > THRESHOLD:
        weight[i] *= MULTIPLIER
    else:
        weight[i] = 1
weight = torch.Tensor(weight)
print(weight)

[492, 472, 322, 78, 0, 0, 1, 0, 0, 496, 492, 457, 305, 37, 426, 318, 107, 9, 443, 387, 216, 42, 9, 50, 25, 4, 1, 0, 0, 0, 10, 2, 0, 139, 29, 3, 1, 88, 17, 1, 0, 0, 0, 11, 3, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 17, 1, 3, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 32, 4, 25, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 218, 31, 136, 178, 2, 15, 17, 48, 1, 22, 37, 0, 0, 0, 7, 0, 4, 7, 0, 0, 0, 3, 0, 2, 3, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 455, 390, 153, 182, 34, 59, 68, 320, 204, 29, 50, 9, 1, 4, 168, 46, 7, 14, 0, 0, 0, 46, 5, 1, 4, 0, 0, 0, 16, 1, 0, 1, 0, 0, 0, 37, 4, 13, 17, 3, 14, 14, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 398, 176, 255, 33, 72, 1, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 496, 496, 425, 427, 50, 0, 0, 0, 0, 6, 134, 87, 1, 0, 11, 